In [1]:
import joblib
import os
from numpy.linalg import inv
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['ANN(58_to_sp162(二合一)).pkl',
 'ANN(58_to_sp162).pkl',
 'ANN(58_to_y162(二合一)).pkl',
 'ANN(58_to_y162).pkl',
 'ANN(energy).pkl',
 'ANN(tray).pkl',
 'eva.pkl',
 'OperationOptimModel.pkl',
 'SVR(4_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(33_to_15).pkl',
 'transformer(43_to_33).pkl',
 'transformer(54_to_23).pkl',
 'transformer(54_to_33).pkl',
 'transformer(SP162_to_Y162).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train(energy).csv', 'train.csv', '現場數據.csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_y162(二合一)).pkl')
#D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
I = joblib.load('./model/transformer(54_to_23).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

,T10,T50,T90,N+A
113,88.7,112.8,144.2,46.61


In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.004806,1.831564,0.333803,3.855244,3.663674,5.32523,0.676762,6.101477,6.021197,14.231356,3.361886,6.131237,6.70851,8.556263,4.971244,3.405689,7.510928,5.015694,2.475905,0.934525,4.221147,0.769865,0.891991


# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.091578,0.183156,1.520198,3.004806,0.036631,0.333803,3.5904,0.073273,3.855244,3.454779,0.576751,0.676762,1.870452,0.63373,0.012642,4.798075,1.317332,6.101477,0.0,7.630844,5.283179,0.067085,3.361886,6.641425,6.131237,8.556263,1.180447,0.669624,1.652801,1.468372,7.320131,0.190797,3.405689,0.035092,3.009279,2.006415,0.149681,0.6428,0.949986,0.698346,0.709843,1.05323,2.458074,0.769865,0.783666,0.8341,0.148216,0.048493,0.002642,0.009398,0.0


# B2 54 to 23

In [9]:
B.W.shape

(23, 54)

# C

In [10]:
C_X = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']+
                                                                   col_names['xna']].tail(1000).reset_index(drop=True)
C_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
0,30.0,10.0,100.0,0.03,0.0,0.0,0.0,0.000000,0.000000,0.003000,0.016000,0.000,0.000000,0.006000,0.000,0.058001,3.804076,0.811016,0.079002,2.059041,0.891018,0.018,6.749135,2.561051,6.881138,0.0,14.834297,10.271206,0.083002,5.885118,8.179164,4.976099,8.707174,1.175024,0.666013,1.645033,1.461029,5.196104,0.135003,2.439049,0.028001,2.625053,1.750035,0.117002,0.504010,0.745015,0.548011,0.365007,0.541011,1.262025,0.434009,0.505010,0.831017,0.096002,0.048001,0.002,0.009000,0.0
1,30.0,2.5,100.0,0.01,0.0,0.0,0.0,0.000000,0.000000,0.003000,0.016000,0.000,0.000000,0.006000,0.000,0.058001,3.804076,0.811016,0.079002,2.059041,0.891018,0.018,6.749135,2.561051,6.881138,0.0,14.834297,10.271206,0.083002,5.885118,8.179164,4.976099,8.707174,1.175024,0.666013,1.645033,1.461029,5.196104,0.135003,2.439049,0.028001,2.625053,1.750035,0.117002,0.504010,0.745015,0.548011,0.365007,0.541011,1.262025,0.434009,0.505010,0.831017,0.096002,0.048001,0.002,0.009000,0.0
2,30.0,5.0,100.0,0.01,0.0,0.0,0.0,0.000000,0.000000,0.003000,0.016000,0.000,0.000000,0.006000,0.000,0.058001,3.804076,0.811016,0.079002,2.059041,0.891018,0.018,6.749135,2.561051,6.881138,0.0,14.834297,10.271206,0.083002,5.885118,8.179164,4.976099,8.707174,1.175024,0.666013,1.645033,1.461029,5.196104,0.135003,2.439049,0.028001,2.625053,1.750035,0.117002,0.504010,0.745015,0.548011,0.365007,0.541011,1.262025,0.434009,0.505010,0.831017,0.096002,0.048001,0.002,0.009000,0.0
3,30.0,10.0,100.0,0.01,0.0,0.0,0.0,0.000000,0.000000,0.003000,0.016000,0.000,0.000000,0.006000,0.000,0.058001,3.804076,0.811016,0.079002,2.059041,0.891018,0.018,6.749135,2.561051,6.881138,0.0,14.834297,10.271206,0.083002,5.885118,8.179164,4.976099,8.707174,1.175024,0.666013,1.645033,1.461029,5.196104,0.135003,2.439049,0.028001,2.625053,1.7

In [11]:
C_X.columns = C.x_col

In [12]:
D_pred,C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

0      54.0
1      54.0
2      54.0
3      54.0
4      54.0
       ... 
995    54.0
996    54.0
997    54.0
998    54.0
999    54.0
Length: 1000, dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

# D

In [13]:
print(D_pred.sum(axis=1))
D_pred

0      300.0
1      300.0
2      300.0
3      300.0
4      300.0
       ...  
995    300.0
996    300.0
997    300.0
998    300.0
999    300.0
Length: 1000, dtype: float32


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# E

In [14]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
463-23,30.0,5.0,100.0,0.3,0.0,0.0,0.0,0.007,0.014,0.114,0.19,0.003,0.015,0.194,0.004,0.386,3.681999,0.784,0.114,1.994,0.861,0.017,6.520998,2.27,6.772999,0.0,13.146998,9.101998,0.063,5.640999,6.189999,4.719999,10.127998,1.19,0.675,1.666,1.48,6.328999,0.165,2.154,0.03,3.490999,2.328,0.126,0.541,0.8,0.588,0.583,0.865,2.02,0.806,0.443,0.656,0.084,0.038,0.001,0.007,0.0


In [15]:
E_pred = E.predict(E_X)
E_pred

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density,rle,rhc,rhe
0,2.135427,2.595214,0.755556,0.716893,0.75512,0.774555,0.019615,0.90051,0.079875


# F

In [16]:
F_X = D_pred[col_names['xhc']]
F_X

,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product Properties_P-Xylene_wt%,Heart Cut Product Properties_M-Xylene_wt%,Heart Cut Product Properties_O-Xylene_wt%,Heart Cut Product Properties_2-Methyloctane_wt%,Heart Cut Product Properties_1-Nonene_wt%,Heart Cut Product Properties_n-Nonane_wt%,Heart Cut Product Properties_i-Propylbenzene_wt%,"Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Heart Cut Product Properties_n-Propylcyclohexane_wt%,Heart Cut Product Properties_n-Propylbenzene_wt%,"Heart Cut Product Properties_1,3,5-Trimethylbenzene_wt%",Heart Cut Product Properties_1-Methyl-2-ethylbenzene_wt%,"Heart Cut Product Properties_1,2,3-Trimethylbenzene_wt%","Heart Cut Product Properties_2,4-Dimethyloctane_wt%","Heart Cut Product Properties_2,2,3,3-Tetramethylhexane_wt%",Heart Cut Product Properties_3-Methylnonane_wt%,Heart Cut Product Properties_tert-Butylcyclohexane_wt%,Heart Cut Product Properties_n-Decane_wt%,"Heart Cut Product Properties_1,2-Diethylbenzene_wt%",Heart Cut Product Properties_n-Undecane_wt%,Heart Cut Product Properties_n-Pentylbenzene_wt%,Heart Cut Product Properties_n-Dodecane_wt%,Heart Cut Product Properties_n-Hexylbenzene_wt%,Heart Cut Product Properties_n-Tridecane_wt%
0,0.0,0.0,0.0,0.000000,0.000000,0.000397,0.001841,0.000000,0.000000,0.002426,0.000000,0.025336,2.274322,0.623641,0.055777,2.112428,0.889588,0.019885,7.433980,2.828260,7.602023,0.0,16.357616,11.338763,0.091686,6.473933,8.973590,5.395077,9.160249,1.221949,0.678572,1.661183,1.365410,4.677119,0.106757,1.651440,0.025431,2.438431,1.060528,0.078955,0.285438,0.444428,0.299402,0.232067,0.320887,0.722157,0.240979,0.272965,0.456717,0.057617,0.032161,0.002105,0.006476,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000192,0.000794,0.000000,0.000000,0.000898,0.000000,0.008116,3.757630,0.768869,0.080527,2.254066,0.955631,0.019789,7.417411,2.815800,7.564441,0.0,16.293642,11.291513,0.091274,6.455398,8.963574,5.421601,9.007392,1.200930,0.651194,1.582516,1.201969,4.064013,0.087828,1.517938,0.015697,2.124007,1.026712,0.071884,0.270248,0.433029,0.287868,0.211695,0.298744,0.723175,0.246440,0.282551,0.443148,0.053949,0.028711,0.001381,0.005815,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000167,0.000683,0.000000,0.000000,0.000832,0.000000,0.008318,3.028738,0.715680,0.077272,2.236488,0.938825,0.019803,7.420519,2.817874,7.571172,0.0,16.304424,11.297694,0.091337,6.457626,8.965093,5.417008,9.109927,1.215106,0.668445,1.646003,1.313124,4.417893,0.094041,1.528715,0.017157,2.304460,1.008777,0.072219,0.260749,0.425753,0.282379,0.212159,0.289957,0.726948,0.244124,0.278220,0.425772,0.054322,0.026505,0.001776,0.005904,0.0
3,0.0,0.0,0.0,0.

In [17]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0       99.999992
1      100.000000
2       99.999989
3       99.999979
4      100.000005
          ...    
995     99.999999
996    100.000003
997    100.000011
998    100.000007
999     99.999998
Length: 1000, dtype: float64


,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.001841,0.000397,0.000000,0.025336,0.002426,4.386750,0.055777,7.602023,8.967095,30.524640,6.473933,5.395077,9.065276,9.160249,1.221949,0.678572,1.661183,1.365410,1.651440,4.783876,3.498959,1.133654,0.332687,1.275110,0.240979,0.495354
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000794,0.000192,0.000000,0.008116,0.000898,6.011695,0.080527,7.564441,9.161700,30.400955,6.455398,5.421601,9.054848,9.007392,1.200930,0.651194,1.582516,1.201969,1.517938,4.151842,3.150719,1.078725,0.337881,1.233614,0.246440,0.477674
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000683,0.000167,0.000000,0.008318,0.000832,5.265226,0.077272,7.571172,9.094827,30.419992,6.457626,5.417008,9.056430,9.109927,1.215106,0.668445,1.646003,1.313124,1.528715,4.511934,3.313237,1.058257,0.334318,1.229064,0.244124,0.458181
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000704,0.000159,0.000000,0.008328,0.000810,4.298969,0.054827,7.588692,8.933047,30.477423,6.466407,5.411312,9.065759,9.236154,1.234621,0.688444,1.683716,1.399243,1.670530,4.900297,3.531880,1.103835,0.318047,1.223686,0.227436,0.475655
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.001990,0.000391,0.000000,0.025295,0.002325,6.340865,0.083505,7.673067,9.344398,30.753499,6.467630,5.178546,8.936161,7.910682,1.050942,0.569329,1.380347,1.122763,1.748039,4.062986,3.247300,1.341938,0.402267,1.482719,0.291349,0.581673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.000072,0.000146,0.000201,0.000018,0.020215,0.008630,0.002252,0.305413,0.165304,0.513549,6.694173,3.340779,3.872089,1.628607,14.525871,3.054234,3.918299,1.505713,10.998332,6.057971,14.800457,11.869679,1.420459,3.168061,0.844978,7.004985,0.267331,1.087014,0.153158,2.772009
996,0.0,0.0,0.0,0.000063,0.000116,0.000159,0.000014,0.021071,0.007162,0.001942,0.287164,0.184251,0.493420,6.620463,3.320008,3.830769,1.618489,14.435402,3.035234,3.893945,1.501202,11.068347,6.154270,15.046222,12.116638,1.440729,3.178789,0.859539,6.762437,0.259347,1.062183,0.146333,2.654292
997,0.0,0.0,0.0,0.000059,0.000131,0.000180,0.000016,0.015987,0.007432,0.002650,0.313035,0.163160,0.518097,6.641361,3.349330,3.871173,1.632872,14.564213,3.062216,3.928589,1.500927,10.932943,5.996068,14.621023,11.828227,1.409865,3.143498,0.834380,7.228485,0.275541,1.109749,0.157857,2.890948
998,0.0,0.0,0.0,0.000050,0.000100,0.000138,0.000013,0.017295,0.006603,0.002307,0.302789,0.173011,0.498856,6.612042,3.342418,3.858835,1.629477,14.533651,3.055828,3.920385,1.507036,11.010736,6.077477,14.850559,11.937316,1.430412,3.173184,0.851893,6.963915,0.264939,1.081829,0.151510,2.745401


# G

In [18]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0       99.999992
1      100.000000
2       99.999989
3       99.999979
4      100.000005
          ...    
995     99.999999
996    100.000003
997    100.000011
998    100.000007
999     99.999998
Length: 1000, dtype: float64


,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10
0,0.0,0.0,0.0,0.000000,0.000000,39.102583,47.811578,0.055777,6.473933,1.221949,0.678572,1.661183,1.365410,1.133654,0.495354
1,0.0,0.0,0.0,0.000000,0.000000,38.453866,48.817202,0.080527,6.455398,1.200930,0.651194,1.582516,1.201969,1.078725,0.477674
2,0.0,0.0,0.0,0.000000,0.000000,38.753468,48.352507,0.077272,6.457626,1.215106,0.668445,1.646003,1.313124,1.058257,0.458181
3,0.0,0.0,0.0,0.000000,0.000000,39.121369,47.771863,0.054827,6.466407,1.234621,0.688444,1.683716,1.399243,1.103835,0.475655
4,0.0,0.0,0.0,0.000000,0.000000,38.858183,48.543695,0.083505,6.467630,1.050942,0.569329,1.380347,1.122763,1.341938,0.581673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.000072,0.000348,20.627845,4.648257,6.694173,14.525871,10.998332,6.057971,14.800457,11.869679,7.004985,2.772009
996,0.0,0.0,0.0,0.000063,0.000275,20.520668,4.620925,6.620463,14.435402,11.068347,6.154270,15.046222,12.116638,6.762437,2.654292
997,0.0,0.0,0.0,0.000059,0.000310,20.649591,4.646783,6.641361,14.564213,10.932943,5.996068,14.621023,11.828227,7.228485,2.890948
998,0.0,0.0,0.0,0.000050,0.000238,20.627541,4.641080,6.612042,14.533651,11.010736,6.077477,14.850559,11.937316,6.963915,2.745401


# H

In [19]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

,"N+2A in Naphtha Feed, wt%","Chlorine in Fresh Catalyst, wt%","Carbon in Spent Catalyst, %","C6P- in Naphtha Feed, wt%","3R211 Inlet Tem, oC","3R212 Inlet Temp, oC","3R213 Inlet Temp, oC","3R214 Inlet Temp, oC",Feed Hydrogen,Feed Methane,Feed Ethane,Feed Propane,Feed n-Butane,Feed i-Butane,Feed C4 Naphthenes,Feed n-Pentane,Feed C5 i-Paraffins,Feed C5 Naphthenes,Feed n-Hexane,Feed C6 i-Paraffins,Feed C6 Naphthenes,Feed Benzene,Feed n-Heptane,Feed C7 i-Paraffins,Feed C7 Naphthenes,Feed Toluene,Feed n-Octane,Feed C8 i-Paraffins,Feed C8 Naphthenes,Feed Ethylbenzene,Feed P-Xylene,Feed M-Xylene,Feed O-Xylene,Feed n-Nonane,Feed C9 i-Paraffins,Feed C9 Naphthenes,Feed C9 Aromtics,Feed C10+ n-Paraffins,Feed C10+ i-Paraffins,Feed C10+ Naphthenes,Feed C10+ Aromtics,"Product Separator Pressure, kg/cm2_g",H2/HC Ratio
0,0.806399,0.050746,0.581421,0.084195,0.161662,0.346598,0.778346,0.71157,0.114761,0.668009,0.99849,0.908725,0.56386,0.295107,0.798495,0.448128,0.648893,0.581986,0.484992,0.666616,0.760565,0.627355,0.628795,0.296419,0.937803,0.558074,0.490782,0.36692,0.047814,0.155091,0.986822,0.421788,0.593192,0.818681,0.925656,0.856164,0.534443,0.065305,0.909885,0.163393,0.772822,0.494054,0.692782


In [20]:
H.predict(x)

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,0.951093,0.190807,1.091748,0.454823,0.147221,0.128028,0.0,0.295226,0.464206,0.002304,0.148346,0.365992,0.014557,0.403996,0.430092,1.509479,0.022132,3.271096,0.147283,0.085827,0.020029,0.834045,0.523302,3.523367,1.293087,0.001007,0.004377,0.00037,2.464159,0.002724,0.008661,0.004466,0.291983


# I

In [21]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.004806,1.831564,0.333803,3.855244,3.663674,5.32523,0.676762,6.101477,6.021197,14.231356,3.361886,6.131237,6.70851,8.556263,4.971244,3.405689,7.510928,5.015694,2.475905,0.934525,4.221147,0.769865,0.891991


In [22]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.091578,0.183156,1.520198,3.004806,0.036631,0.333803,3.5904,0.073273,3.855244,3.454779,0.576751,0.676762,1.870452,0.63373,0.012642,4.798075,1.317332,6.101477,0.0,7.630844,5.283179,0.067085,3.361886,6.641425,6.131237,8.556263,1.180447,0.669624,1.652801,1.468372,7.320131,0.190797,3.405689,0.035092,3.009279,2.006415,0.149681,0.6428,0.949986,0.698346,0.709843,1.05323,2.458074,0.769865,0.783666,0.8341,0.148216,0.048493,0.002642,0.009398,0.0


In [23]:
I(B_pred)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.004806,1.831564,0.333803,3.855244,3.663674,5.32523,0.676762,6.101477,6.021197,14.231356,3.361886,6.131237,6.70851,8.556263,4.971244,3.405689,7.510928,5.015694,2.475905,0.934525,4.221147,0.769865,0.891991


# 把全部模組整合到一塊

In [24]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_y162(二合一)).pkl')# 預測wt%同時給出分離係數
        #self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl') #不需要了
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.I = joblib.load('./model/transformer(54_to_23).pkl')
        
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

    def __call__(self, X):
        x4, case4 = X.iloc[:, :4], X.iloc[:, 4:]
        # model forward
        self.y23 = self.A.predict(x4)  # percent sum = 100
        self.Xna = self.B(self.y23)  # percent sum = 100
        # percent sum = 1 * 54component = 54
        c_x = case4.join(self.Xna)
        c_x.columns = self.C.x_col
        self.y162,self.sp162 = self.C.predict(c_x)
        
        self.xhc = self.y162[self.col_names['xhc']]  # percent sum = 100
        self.xhc33_p = self.F(self.xhc)/100  # percent

        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:, :2]
        self.density = E_pred.iloc[:, 2:]

        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]

        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:, 2].values

        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))

        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton

        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*100000/self.fna_ton
        self.fhc_ton = self.fhc_ton*100000/self.fna_ton
        self.fhe_ton = self.fhe_ton*100000/self.fna_ton

        # add columns name
        self.fle_ton = pd.Series(
            self.fle_ton, name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(
            self.fhe_ton, name=self.col_names['Rate_ton'][3])

        # scale duty
        self.duty *= 100/self.fna_ton[0]

        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        self.xhc33_tmp = self.xhc33*74000/self.fhc_ton[0]

        # xhe33 = H(someting + xhc33 + something)
        xhc23 = self.I(self.xhc)
        C6Pm = xhc23[['C5NP', 'C5IP', 'C6IP', 'C6NP']].sum(axis=1)[0]

        NpA = xhc23[['C5N', 'C6N', 'C7N', 'C8N', 'C9N', 'C10N']].sum(axis=1)[0] +\
            xhc23[['C6A', 'C7A', 'C8A', 'C9A', 'C10A']].sum(axis=1)[0]*2

        self.H_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + \
            self.xhc33_tmp.values.ravel().tolist()+[4.798, 1.44]
        self.H_input = np.array([self.H_input])
        self.H_input = pd.DataFrame(self.H_input, columns=self.H.x_col)
        self.reformer_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + self.xhc33.values.ravel().tolist() + [4.798, 1.44]
        self.reformer_input = np.array([self.reformer_input])
        self.reformer_input = pd.DataFrame(self.reformer_input, columns=self.H.x_col)

        self.重組33_tmp = self.H.predict(self.H_input)

        self.重組33 = self.重組33_tmp *self.fhc_ton[0]/74000

        # final output
        self.重組15 = self.G(self.重組33)

        # raname and combine return to user
        self.predict = self.重組15.join(self.fle_ton).join(
            self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.重組33

In [25]:
eva = EVA()

# save eva

In [26]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [27]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [28]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1)

In [29]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

,T10,T50,T90,N+A,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%
0,88.7,112.8,144.2,46.61,10.0,2.5,100.0,2.0


# get eva result

In [30]:
eva(X)

In [31]:
eva.predict

,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10,Light End Product Properties_Light End Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr
0,2731.974486,790.770378,2285.409514,2386.891459,2090.139243,13956.189275,652.730569,3067.194811,14784.762811,3565.624,2355.415135,10479.815784,5352.496865,10694.025514,1935.619459,14274.213289,8596.730617,2.621242,4.696178


In [32]:
eva.xhc33

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.148288,1.248946,1.717301,0.156118,118.667435,56.122238,22.521498,926.096702,438.42417,2817.266235,316.699472,6107.60439,5657.606725,14242.339591,3366.238476,6137.336093,6717.423389,8357.168636,1151.17328,622.240361,1518.991069,1181.23525,2294.836493,5774.107624,3662.254759,1538.912322,545.011992,2588.552221,462.854531,504.097088


In [33]:
eva.naphtha

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.004806,1.831564,0.333803,3.855244,3.663674,5.32523,0.676762,6.101477,6.021197,14.231356,3.361886,6.131237,6.70851,8.556263,4.971244,3.405689,7.510928,5.015694,2.475905,0.934525,4.221147,0.769865,0.891991


In [34]:
eva.pre_d

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

In [35]:
eva.reform

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,2731.974486,790.770378,2285.409514,2386.891459,1018.508541,1071.630703,1.397368,2068.597838,3129.771676,51.414652,905.177135,2093.765838,126.582189,3067.194811,921.554486,3599.786378,169.229581,14784.762811,219.861919,610.513405,162.590284,3565.624,2355.415135,10479.815784,5352.496865,14.103117,109.667392,14.544796,10694.025514,99.594628,182.398095,128.369068,1935.619459
